In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.seqsignet_full_attention_functions import seqsignet_full_attention_hyperparameter_search

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [6]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-10-03 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-10-03 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-10-03 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-10-03 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-10-03 00:00:34,-1


In [7]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

In [8]:
features = ["timeline_index", "speaker"]
standardise_method = [None, None]
include_features_in_path = True
include_features_in_input = False

In [9]:
num_epochs = 100
dimensions = [15]  # [50, 15]
swmhau_parameters = [(12, 3, 10), (8, 4, 6)]#, (8, 4, 12)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256,256],[512,512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [10]:
kwargs = {
    "num_epochs": num_epochs,
    "df": anno_mi,
    "id_column": "transcript_id",
    "label_column": "client_talk_type",
    "embeddings": sbert_embeddings,
    "y_data": y_data_client,
    "output_dim": output_dim_client,
    "dimensions": dimensions,
    "log_signature": True,
    "swmhau_parameters": swmhau_parameters,
    "num_layers": num_layers,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# history_length=11

In [11]:
shift = 3
window_size = 5
n = 3

In [12]:
(
    seqsignet_attention_mha_umap_kfold_11,
    best_seqsignet_attention_mha_umap_kfold_11,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_mha_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_umap_focal_2_3_5_3_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_umap_focal_2_3_5_3_kfold_best_model.csv


In [13]:
seqsignet_attention_mha_umap_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.645594,0.515260,"[0.7709431751611013, 0.4511840465309514, 0.323...",0.511037,"[0.7810089020771513, 0.45439330543933054, 0.29...",0.521233,"[0.7611336032388664, 0.44801980198019803, 0.35...",None,0.658478,...,1,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.641188,0.517020,"[0.7687997640813918, 0.4445414847161572, 0.337...",0.512954,"[0.7842960288808665, 0.47217068645640076, 0.28...",0.531290,"[0.753903990746096, 0.41996699669966997, 0.42]",None,0.667148,...,12,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.649808,0.517789,"[0.775892334698654, 0.453377538334024, 0.32409...",0.513875,"[0.7850799289520426, 0.45545378850957535, 0.30...",0.523049,"[0.7669172932330827, 0.4513201320132013, 0.350...",None,0.669236,...,123,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.620881,0.442699,"[0.7602836879432624, 0.3857949959644875, 0.182...",0.450984,"[0.7461024498886414, 0.37756714060031593, 0.22...",0.440104,"[0.7750144592249856, 0.3943894389438944, 0.150...",None,0.644348,...,1,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.602682,0.444819,"[0.7475183426845058, 0.34862385321100914, 0.23...",0.445505,"[0.743773260807329, 0.3524451939291737, 0.2402...",0.444183,"[0.7513013302486987, 0.3448844884488449, 0.236...",None,0.628934,...,12,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.588123,0.453424,"[0.7287290312830588, 0.38890970400899216, 0.24...",0.450405,"[0.7632162076606521, 0.3562113932738504, 0.231...",0.459996,"[0.6972238288027762, 0.4282178217821782, 0.254...",None,0.621066,...,12,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.616858,0.458677,"[0.7562018819503848, 0.35914065666801775, 0.26...",0.468131,"[0.7457817772778402, 0.3529880478087649, 0.305...",0.453234,"[0.7669172932330827, 0.3655115511551155, 0.227...",None,0.642100,...,123,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.642146,0.504475,"[0.7761925349473987, 0.40241075567918405, 0.33...",0.505421,"[0.7736282677391554, 0.45925925925925926, 0.28...",0.515317,"[0.7787738577212261, 0.3580858085808581, 0.409...",None,0.669236,...,1,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.646552,0.521250,"[0.7721592560302238, 0.4274120829576195, 0.364...",0.518676,"[0.7759929906542056, 0.4711729622266402, 0.308...",0.534363,"[0.7683632157316368, 0.3910891089108911, 0.443...",None,0.662653,...,12,focal,2,True,5,Conv1d,None,concatenation,64,23


In [14]:
best_seqsignet_attention_mha_umap_kfold_11["f1"].mean()

0.5011317343319162

In [15]:
best_seqsignet_attention_mha_umap_kfold_11["precision"].mean()

0.501423817705441

In [16]:
best_seqsignet_attention_mha_umap_kfold_11["recall"].mean()

0.5073380601989252

In [17]:
np.stack(best_seqsignet_attention_mha_umap_kfold_11["f1_scores"]).mean(axis=0)

array([0.76704038, 0.41825766, 0.31809716])

In [18]:
np.stack(best_seqsignet_attention_mha_umap_kfold_11["precision_scores"]).mean(axis=0)

array([0.76868738, 0.44003164, 0.29555244])

In [19]:
np.stack(best_seqsignet_attention_mha_umap_kfold_11["recall_scores"]).mean(axis=0)

array([0.76576056, 0.39988999, 0.35636364])

## GRP

In [20]:
(
    seqsignet_attention_mha_grp_kfold_11,
    best_seqsignet_attention_mha_grp_kfold_11,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_attention_mha_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_grp_focal_2_3_5_3_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_grp_focal_2_3_5_3_kfold_best_model.csv


In [21]:
seqsignet_attention_mha_grp_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.635057,0.518282,"[0.7630833457581632, 0.437609841827768, 0.3541...",0.513375,"[0.7876269621421976, 0.4680451127819549, 0.284...",0.540002,"[0.7400231347599768, 0.41089108910891087, 0.46...",None,0.655588,...,1,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.633908,0.514896,"[0.7631931024230713, 0.4353680430879712, 0.346...",0.512495,"[0.7852554297950444, 0.47736220472440943, 0.27...",0.536591,"[0.7423366107576634, 0.40016501650165015, 0.46...",None,0.659441,...,12,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.640805,0.514914,"[0.7685103892303191, 0.43027888446215135, 0.34...",0.511458,"[0.7778436018957346, 0.46418338108882523, 0.29...",0.528008,"[0.7593984962406015, 0.400990099009901, 0.4236...",None,0.667148,...,123,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.624904,0.457209,"[0.7601874201334659, 0.38722554890219557, 0.22...",0.471406,"[0.7467224546722455, 0.3750966744006187, 0.292...",0.452043,"[0.7741469057258531, 0.40016501650165015, 0.18...",None,0.657514,...,1,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.631801,0.480894,"[0.7631238314396663, 0.4079522862823061, 0.271...",0.488529,"[0.7590844062947067, 0.39370683039140447, 0.31...",0.476825,"[0.7672064777327935, 0.42326732673267325, 0.24]",None,0.665061,...,12,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.628352,0.479312,"[0.7651869158878505, 0.4085603112840467, 0.264...",0.481825,"[0.7728613569321534, 0.3865979381443299, 0.286...",0.478762,"[0.7576633892423366, 0.43316831683168316, 0.24...",None,0.655267,...,12,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.600192,0.438723,"[0.7411627227578147, 0.37676842889054357, 0.19...",0.447833,"[0.7488193624557261, 0.34328358208955223, 0.25...",0.438263,"[0.7336610757663389, 0.4174917491749175, 0.163...",None,0.651252,...,123,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.634674,0.520512,"[0.7608662729062547, 0.45924132364810333, 0.34...",0.511650,"[0.7987281399046104, 0.44944707740916273, 0.28...",0.539241,"[0.7264314632735686, 0.46947194719471946, 0.42...",None,0.655267,...,1,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.645019,0.518685,"[0.7718890663569045, 0.4218678815489749, 0.362...",0.516872,"[0.7751531058617673, 0.47100712105798576, 0.30...",0.532646,"[0.7686524002313476, 0.382013201320132, 0.4472...",None,0.665864,...,12,focal,2,True,5,Conv1d,None,concatenation,64,23


In [22]:
best_seqsignet_attention_mha_grp_kfold_11["f1"].mean()

0.5132267886107895

In [23]:
best_seqsignet_attention_mha_grp_kfold_11["precision"].mean()

0.5087124344063694

In [24]:
best_seqsignet_attention_mha_grp_kfold_11["recall"].mean()

0.5236764343486594

In [25]:
np.stack(best_seqsignet_attention_mha_grp_kfold_11["f1_scores"]).mean(axis=0)

array([0.76442744, 0.44165738, 0.33359554])

In [26]:
np.stack(best_seqsignet_attention_mha_grp_kfold_11["precision_scores"]).mean(axis=0)

array([0.77840999, 0.4535798 , 0.29414751])

In [27]:
np.stack(best_seqsignet_attention_mha_grp_kfold_11["recall_scores"]).mean(axis=0)

array([0.75130133, 0.43124312, 0.38848485])

# history_length=20

In [28]:
shift = 3
window_size = 5
n = 6

In [29]:
(
    seqsignet_attention_mha_umap_kfold_20,
    best_seqsignet_attention_mha_umap_kfold_20,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_mha_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_umap_focal_2_3_5_6_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_umap_focal_2_3_5_6_kfold_best_model.csv


In [30]:
seqsignet_attention_mha_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.637356,0.516129,"[0.7654650645304851, 0.44117647058823534, 0.34...",0.510886,"[0.7858665854401462, 0.4636363636363636, 0.283...",0.532599,"[0.746096009253904, 0.4207920792079208, 0.4309...",None,0.668433,...,1,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.636782,0.513819,"[0.7676232523914643, 0.4186691312384473, 0.355...",0.513291,"[0.7815403056637699, 0.47584033613445376, 0.28...",0.535379,"[0.7541931752458069, 0.37376237623762376, 0.47...",None,0.660405,...,12,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.637356,0.515270,"[0.7667109340416113, 0.4293429342934293, 0.349...",0.512624,"[0.7827658933413679, 0.47227722772277225, 0.28...",0.534349,"[0.7513013302486987, 0.3935643564356436, 0.458...",None,0.661207,...,123,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.597126,0.432631,"[0.7443189938545091, 0.33028286189683853, 0.22...",0.435358,"[0.7358010737496468, 0.33305369127516776, 0.23...",0.430501,"[0.7530364372469636, 0.32755775577557755, 0.21...",None,0.635838,...,1,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.597701,0.451603,"[0.7373244641537325, 0.3819628647214854, 0.235...",0.452792,"[0.7541578469912307, 0.35318850735809393, 0.25...",0.452962,"[0.7212261422787739, 0.4158415841584158, 0.221...",None,0.640013,...,12,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.615709,0.460318,"[0.7609350351355227, 0.35045317220543803, 0.26...",0.460173,"[0.7547652916073969, 0.36742081447963804, 0.25...",0.461336,"[0.7672064777327935, 0.334983498349835, 0.2818...",None,0.664258,...,12,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.624138,0.454414,"[0.7642140468227424, 0.34387527839643656, 0.25...",0.462783,"[0.7374932759548144, 0.3736689254598258, 0.277...",0.449263,"[0.7929438982070561, 0.3184818481848185, 0.236...",None,0.662813,...,123,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.653831,0.508982,"[0.7829787234042553, 0.41834862385321103, 0.32...",0.512644,"[0.7683741648106904, 0.47107438016528924, 0.29...",0.510856,"[0.7981492192018508, 0.37623762376237624, 0.35...",None,0.664740,...,1,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.641379,0.520587,"[0.7706666666666666, 0.4329718004338395, 0.358...",0.514548,"[0.790097205346294, 0.4565416285452882, 0.2970...",0.538265,"[0.7521688837478311, 0.41171617161716173, 0.45...",None,0.663134,...,12,focal,2,True,5,Conv1d,None,concatenation,64,23


In [31]:
best_seqsignet_attention_mha_umap_kfold_20["f1"].mean()

0.5043913067188163

In [32]:
best_seqsignet_attention_mha_umap_kfold_20["precision"].mean()

0.4963152847177106

In [33]:
best_seqsignet_attention_mha_umap_kfold_20["recall"].mean()

0.5235104559231708

In [34]:
np.stack(best_seqsignet_attention_mha_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.74594372, 0.44552708, 0.32170312])

In [35]:
np.stack(best_seqsignet_attention_mha_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.79084425, 0.42861956, 0.26948205])

In [36]:
np.stack(best_seqsignet_attention_mha_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.70628494, 0.46424642, 0.4       ])

## GRP

In [37]:
(
    seqsignet_attention_mha_grp_kfold_20,
    best_seqsignet_attention_mha_grp_kfold_20,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_attention_mha_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_grp_focal_2_3_5_6_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_grp_focal_2_3_5_6_kfold_best_model.csv


In [38]:
seqsignet_attention_mha_grp_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.643295,0.518670,"[0.7702484198147875, 0.4408834993503681, 0.344...",0.513865,"[0.7832585949177877, 0.4639927073837739, 0.294...",0.531331,"[0.7576633892423366, 0.41996699669966997, 0.41...",None,0.667951,...,1,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.637548,0.507900,"[0.7676974937710686, 0.4297520661157025, 0.326...",0.504239,"[0.7783060921248143, 0.45446182152713893, 0.27...",0.518625,"[0.7573742047426258, 0.4075907590759076, 0.390...",None,0.664258,...,12,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.638697,0.525275,"[0.760179640718563, 0.46192259675405745, 0.353...",0.517138,"[0.7880198634388579, 0.4659949622166247, 0.297...",0.542841,"[0.7342394447657605, 0.45792079207920794, 0.43...",None,0.661529,...,123,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.615326,0.471926,"[0.7521994134897361, 0.3912037037037037, 0.272...",0.474406,"[0.762938726948245, 0.3673913043478261, 0.2928...",0.471540,"[0.7417582417582418, 0.4183168316831683, 0.254...",None,0.664098,...,1,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.629119,0.489164,"[0.7648866130212143, 0.4071515644047135, 0.295...",0.486564,"[0.7740598164050932, 0.4011208967173739, 0.284...",0.492189,"[0.7559282822440717, 0.41336633663366334, 0.30...",None,0.667791,...,12,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.634674,0.486503,"[0.7690763052208835, 0.39797211660329534, 0.29...",0.487645,"[0.7629482071713147, 0.4077922077922078, 0.292...",0.485548,"[0.7753036437246964, 0.3886138613861386, 0.292...",None,0.679994,...,12,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.603831,0.483467,"[0.7378141083863153, 0.4019870080244554, 0.310...",0.477293,"[0.7741423125794155, 0.37437722419928826, 0.28...",0.494124,"[0.7047426257952574, 0.43399339933993397, 0.34...",None,0.656872,...,123,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.633716,0.519062,"[0.7587659894657637, 0.44899598393574297, 0.34...",0.510138,"[0.79102604330091, 0.43740219092331767, 0.3019...",0.534934,"[0.7290341237709659, 0.4612211221122112, 0.414...",None,0.663455,...,1,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.605556,0.501166,"[0.7358693978130294, 0.4101346001583532, 0.357...",0.490995,"[0.787149917627677, 0.3942161339421613, 0.2916...",0.526691,"[0.6908617698091383, 0.4273927392739274, 0.461...",None,0.643385,...,12,focal,2,True,5,Conv1d,None,concatenation,64,23


In [39]:
best_seqsignet_attention_mha_grp_kfold_20["f1"].mean()

0.5033058602955406

In [40]:
best_seqsignet_attention_mha_grp_kfold_20["precision"].mean()

0.4993054531774514

In [41]:
best_seqsignet_attention_mha_grp_kfold_20["recall"].mean()

0.5093960878640399

In [42]:
np.stack(best_seqsignet_attention_mha_grp_kfold_20["f1_scores"]).mean(axis=0)

array([0.76246417, 0.42082179, 0.32663162])

In [43]:
np.stack(best_seqsignet_attention_mha_grp_kfold_20["precision_scores"]).mean(axis=0)

array([0.77152897, 0.42556551, 0.30082188])

In [44]:
np.stack(best_seqsignet_attention_mha_grp_kfold_20["recall_scores"]).mean(axis=0)

array([0.75361481, 0.41639164, 0.35818182])

# history_length=35

In [45]:
shift = 3
window_size = 5
n = 11

In [46]:
(
    seqsignet_attention_mha_umap_kfold_35,
    best_seqsignet_attention_mha_umap_kfold_35,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_mha_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_umap_focal_2_3_5_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_umap_focal_2_3_5_11_kfold_best_model.csv


In [47]:
seqsignet_attention_mha_umap_kfold_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.628927,0.514185,"[0.7564954682779456, 0.45226130653266333, 0.33...",0.507354,"[0.7919038583175205, 0.45918367346938777, 0.27...",0.534736,"[0.7241179872758821, 0.44554455445544555, 0.43...",None,0.655267,...,1,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.640805,0.505663,"[0.7727730608720678, 0.39704524469067404, 0.34...",0.505543,"[0.7691205958178172, 0.45073375262054505, 0.29...",0.516476,"[0.7764603817235396, 0.3547854785478548, 0.418...",None,0.664900,...,12,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.637548,0.507873,"[0.7660690861390468, 0.4252315835906484, 0.332...",0.505074,"[0.772259770790479, 0.45687203791469194, 0.286...",0.518010,"[0.7599768652400232, 0.3976897689768977, 0.396...",None,0.666988,...,123,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.592912,0.445793,"[0.7361029842012873, 0.3535514764565044, 0.247...",0.445115,"[0.7448194197750148, 0.34234930448222567, 0.24...",0.446791,"[0.7275882012724117, 0.3655115511551155, 0.247...",None,0.639210,...,1,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.581034,0.442933,"[0.7201820940819423, 0.3887303851640514, 0.219...",0.443951,"[0.7576628352490421, 0.34233668341708545, 0.23...",0.448332,"[0.6862348178137652, 0.44966996699669964, 0.20...",None,0.642421,...,12,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.592912,0.451721,"[0.7363568773234199, 0.36880522474068383, 0.25]",0.450096,"[0.7578818487909397, 0.34507548526240117, 0.24...",0.454929,"[0.7160208212839791, 0.39603960396039606, 0.25...",None,0.652698,...,12,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.602490,0.454492,"[0.7497792169561377, 0.35833657209483094, 0.25...",0.454720,"[0.7634892086330936, 0.3387215282880235, 0.261...",0.455336,"[0.736552920763447, 0.3803630363036304, 0.2490...",None,0.647238,...,123,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.633333,0.510547,"[0.7560644869670031, 0.46907020872865274, 0.30...",0.504393,"[0.7892418999685435, 0.4342937456078707, 0.289...",0.520306,"[0.7255639097744361, 0.5099009900990099, 0.325...",None,0.662653,...,1,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.614368,0.496748,"[0.7503819126183928, 0.4233055885850179, 0.316...",0.488990,"[0.7953367875647669, 0.4073226544622426, 0.264...",0.515126,"[0.7102371312897628, 0.4405940594059406, 0.394...",None,0.645633,...,12,focal,2,True,5,Conv1d,None,concatenation,64,23


In [48]:
best_seqsignet_attention_mha_umap_kfold_35["f1"].mean()

0.5139245284076588

In [49]:
best_seqsignet_attention_mha_umap_kfold_35["precision"].mean()

0.5074300394809725

In [50]:
best_seqsignet_attention_mha_umap_kfold_35["recall"].mean()

0.5323878132945356

In [51]:
np.stack(best_seqsignet_attention_mha_umap_kfold_35["f1_scores"]).mean(axis=0)

array([0.75719447, 0.44867743, 0.33590168])

In [52]:
np.stack(best_seqsignet_attention_mha_umap_kfold_35["precision_scores"]).mean(axis=0)

array([0.7946156 , 0.44311686, 0.28455766])

In [53]:
np.stack(best_seqsignet_attention_mha_umap_kfold_35["recall_scores"]).mean(axis=0)

array([0.72315404, 0.45764576, 0.41636364])

## GRP

In [54]:
(
    seqsignet_attention_mha_grp_kfold_35,
    best_seqsignet_attention_mha_grp_kfold_35,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_attention_mha_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_grp_focal_2_3_5_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_grp_focal_2_3_5_11_kfold_best_model.csv


In [55]:
seqsignet_attention_mha_grp_kfold_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.642720,0.515170,"[0.7744117647058822, 0.4314765694076039, 0.339...",0.511741,"[0.7878515858767206, 0.46476190476190476, 0.28...",0.529839,"[0.7614227877385772, 0.40264026402640263, 0.42...",None,0.660886,...,1,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.649042,0.523271,"[0.7753368482718218, 0.44658493870402804, 0.34...",0.519374,"[0.785459940652819, 0.47574626865671643, 0.296...",0.535421,"[0.7654713707345286, 0.4207920792079208, 0.42]",None,0.664900,...,12,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.639464,0.513681,"[0.7644444444444445, 0.4490118577075099, 0.327...",0.508717,"[0.7837181044957473, 0.4309559939301973, 0.311...",0.520066,"[0.746096009253904, 0.46864686468646866, 0.345...",None,0.661047,...,123,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.619157,0.484846,"[0.7585596221959859, 0.3886606409202958, 0.307...",0.479858,"[0.7745629897528632, 0.38707037643207853, 0.27...",0.492368,"[0.7432041642567958, 0.39026402640264024, 0.34...",None,0.672608,...,1,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.634674,0.493259,"[0.7648177959900483, 0.4251908396946565, 0.289...",0.499017,"[0.7742222222222223, 0.3955965909090909, 0.327...",0.491737,"[0.7556390977443609, 0.45957095709570955, 0.26]",None,0.662974,...,12,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.635441,0.497213,"[0.7609567671965533, 0.4575018477457502, 0.273...",0.499738,"[0.7824625725633975, 0.4143239625167336, 0.302...",0.500139,"[0.7406015037593985, 0.5107260726072608, 0.249...",None,0.672608,...,12,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.618582,0.492300,"[0.7552238805970148, 0.3984095427435387, 0.323...",0.486071,"[0.7803824799506478, 0.3844973138910207, 0.293...",0.501668,"[0.7316367842683632, 0.41336633663366334, 0.36]",None,0.681599,...,123,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.618582,0.480782,"[0.7558960492714436, 0.4207811348563007, 0.265...",0.480086,"[0.7864957799312285, 0.3801597869507324, 0.273...",0.485631,"[0.7275882012724117, 0.47112211221122113, 0.25...",None,0.645311,...,1,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.629119,0.505617,"[0.7606635071090048, 0.416452442159383, 0.3397...",0.499608,"[0.7795992714025501, 0.43315508021390375, 0.28...",0.520599,"[0.7426257952573742, 0.400990099009901, 0.4181...",None,0.650771,...,12,focal,2,True,5,Conv1d,None,concatenation,64,23


In [56]:
best_seqsignet_attention_mha_grp_kfold_35["f1"].mean()

0.5040143830156835

In [57]:
best_seqsignet_attention_mha_grp_kfold_35["precision"].mean()

0.4997326393070261

In [58]:
best_seqsignet_attention_mha_grp_kfold_35["recall"].mean()

0.5098287852691188

In [59]:
np.stack(best_seqsignet_attention_mha_grp_kfold_35["f1_scores"]).mean(axis=0)

array([0.76211721, 0.42877463, 0.32115131])

In [60]:
np.stack(best_seqsignet_attention_mha_grp_kfold_35["precision_scores"]).mean(axis=0)

array([0.77753087, 0.41798238, 0.30368467])

In [61]:
np.stack(best_seqsignet_attention_mha_grp_kfold_35["recall_scores"]).mean(axis=0)

array([0.74734914, 0.44031903, 0.34181818])

# history_length=80

In [11]:
shift = 3
window_size = 5
n = 26

In [63]:
(
    seqsignet_attention_mha_umap_kfold_80,
    best_seqsignet_attention_mha_umap_kfold_80,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_mha_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 26: history length = 80
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 26: history length = 80
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_umap_focal_2_3_5_26_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_umap_focal_2_3_5_26_kfold_best_model.csv


In [64]:
seqsignet_attention_mha_umap_kfold_80

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.642146,0.517118,"[0.768294493097818, 0.4563977180114099, 0.3266...",0.510665,"[0.7892650198231168, 0.45088566827697263, 0.29...",0.527122,"[0.7484094852515906, 0.46204620462046203, 0.37...",None,0.662492,...,1,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.625862,0.511835,"[0.7565938206480783, 0.43412908930150307, 0.34...",0.508516,"[0.7900535096002518, 0.4676190476190476, 0.267...",0.538202,"[0.7258530942741469, 0.4051155115511551, 0.483...",None,0.659120,...,12,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.632759,0.508760,"[0.76156897311591, 0.4218540080609046, 0.34285...",0.505876,"[0.7739623768289041, 0.46131243878550443, 0.28...",0.524848,"[0.7495662232504338, 0.3886138613861386, 0.436...",None,0.653500,...,123,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.594636,0.430982,"[0.7464688257954061, 0.31009830205540667, 0.23...",0.431395,"[0.7366938890453394, 0.3382066276803119, 0.219...",0.433058,"[0.7565066512434934, 0.2863036303630363, 0.256...",None,0.662653,...,1,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.591379,0.458480,"[0.7251655629139072, 0.3969410050983248, 0.253...",0.459134,"[0.756120527306968, 0.35528031290743156, 0.266]",0.462711,"[0.6966454598033546, 0.44966996699669964, 0.24...",None,0.662813,...,12,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.572605,0.444621,"[0.7224018475750578, 0.3563926113726911, 0.255...",0.442756,"[0.7724728350345736, 0.31762427372498386, 0.23...",0.452971,"[0.6784268363215732, 0.40594059405940597, 0.27...",None,0.663616,...,12,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.573755,0.443745,"[0.7237921604375569, 0.34314100114635077, 0.26...",0.439798,"[0.762483994878361, 0.3195729537366548, 0.2373...",0.452494,"[0.6888374783111625, 0.3704620462046205, 0.298...",None,0.648523,...,123,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.619732,0.496147,"[0.7490568884864948, 0.4351116155883466, 0.304...",0.490709,"[0.7832123698327548, 0.4018169112508735, 0.287...",0.505272,"[0.7177559282822441, 0.4744224422442244, 0.323...",None,0.668593,...,1,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.639655,0.486469,"[0.7691015952980689, 0.39013035381750466, 0.30...",0.493786,"[0.745119305856833, 0.44764957264957267, 0.288...",0.484372,"[0.794679005205321, 0.3457095709570957, 0.3127...",None,0.667951,...,12,focal,2,True,5,Conv1d,None,concatenation,64,23


In [65]:
best_seqsignet_attention_mha_umap_kfold_80["f1"].mean()

0.4993284496130104

In [66]:
best_seqsignet_attention_mha_umap_kfold_80["precision"].mean()

0.4933308735861121

In [67]:
best_seqsignet_attention_mha_umap_kfold_80["recall"].mean()

0.5126887375454338

In [68]:
np.stack(best_seqsignet_attention_mha_umap_kfold_80["f1_scores"]).mean(axis=0)

array([0.75471268, 0.41193288, 0.33133979])

In [69]:
np.stack(best_seqsignet_attention_mha_umap_kfold_80["precision_scores"]).mean(axis=0)

array([0.77653191, 0.41468024, 0.28878048])

In [70]:
np.stack(best_seqsignet_attention_mha_umap_kfold_80["recall_scores"]).mean(axis=0)

array([0.73433584, 0.40979098, 0.39393939])

## GRP

In [12]:
(
    seqsignet_attention_mha_grp_kfold_80,
    best_seqsignet_attention_mha_grp_kfold_80,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_attention_mha_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 26: history length = 80
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 26: history length = 80
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_grp_focal_2_3_5_26_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_grp_focal_2_3_5_26_kfold_best_model.csv


In [13]:
seqsignet_attention_mha_grp_kfold_80

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.630268,0.508910,"[0.7621597501115572, 0.4240561896400351, 0.340...",0.504457,"[0.7846860643185298, 0.4530956848030019, 0.275...",0.528287,"[0.7408906882591093, 0.39851485148514854, 0.44...",None,0.677264,...,1,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.613985,0.505207,"[0.7441646191646192, 0.4340080971659919, 0.337...",0.496795,"[0.7933857236411264, 0.4260731319554849, 0.270...",0.530070,"[0.7006940427993059, 0.44224422442244227, 0.44...",None,0.648523,...,12,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.636590,0.516778,"[0.7631151174818974, 0.4347826086956521, 0.352...",0.511955,"[0.7802961619824721, 0.4647887323943662, 0.290...",0.534121,"[0.7466743782533256, 0.4084158415841584, 0.447...",None,0.662492,...,123,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.600000,0.472438,"[0.7364945978391355, 0.39464356045687277, 0.28...",0.466875,"[0.7654398003742982, 0.3775433308214017, 0.257...",0.481614,"[0.7096587622903412, 0.41336633663366334, 0.32...",None,0.692678,...,1,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.609004,0.469126,"[0.7442485808186435, 0.40785169029443835, 0.25...",0.473231,"[0.7697775030902348, 0.3645224171539961, 0.285...",0.471380,"[0.7203585887796414, 0.4628712871287129, 0.230...",None,0.683365,...,12,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.617433,0.486651,"[0.7539964476021314, 0.3944805194805195, 0.311...",0.481349,"[0.7722862340812614, 0.38817891373801916, 0.28...",0.494333,"[0.736552920763447, 0.400990099009901, 0.34545...",None,0.692518,...,12,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.601724,0.476767,"[0.7348711803475134, 0.39386973180076634, 0.30...",0.472674,"[0.7622747047855811, 0.3676680972818312, 0.288...",0.483275,"[0.7093695777906304, 0.4240924092409241, 0.316...",None,0.686416,...,123,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.629885,0.500164,"[0.7668621700879764, 0.3877005347593583, 0.345...",0.495819,"[0.7778108268887567, 0.42151162790697677, 0.28...",0.515952,"[0.7562174667437825, 0.3589108910891089, 0.432...",None,0.666346,...,1,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.600958,0.487241,"[0.728066074489637, 0.43860255966793493, 0.295...",0.484668,"[0.7894558972625887, 0.3776057176891007, 0.286...",0.500758,"[0.675534991324465, 0.523102310231023, 0.30363...",None,0.632145,...,12,focal,2,True,5,Conv1d,None,concatenation,64,23


In [14]:
best_seqsignet_attention_mha_grp_kfold_80["f1"].mean()

0.49920963635111076

In [15]:
best_seqsignet_attention_mha_grp_kfold_80["precision"].mean()

0.49420823827128735

In [16]:
best_seqsignet_attention_mha_grp_kfold_80["recall"].mean()

0.5070538850684759

In [17]:
np.stack(best_seqsignet_attention_mha_grp_kfold_80["f1_scores"]).mean(axis=0)

array([0.75277997, 0.41648313, 0.32836581])

In [18]:
np.stack(best_seqsignet_attention_mha_grp_kfold_80["precision_scores"]).mean(axis=0)

array([0.77609221, 0.39747913, 0.30905338])

In [19]:
np.stack(best_seqsignet_attention_mha_grp_kfold_80["recall_scores"]).mean(axis=0)

array([0.73086563, 0.43756876, 0.35272727])

# history_length=110

In [20]:
shift = 3
window_size = 5
n = 36

In [21]:
(
    seqsignet_attention_mha_umap_kfold_110,
    best_seqsignet_attention_mha_umap_kfold_110,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_mha_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 36: history length = 110
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 36: history length = 110
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_umap_focal_2_3_5_36_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_umap_focal_2_3_5_36_kfold_best_model.csv


In [22]:
seqsignet_attention_mha_umap_kfold_110

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.653257,0.523732,"[0.778003482298317, 0.450378282153983, 0.34281...",0.522183,"[0.7807221898660455, 0.4888888888888889, 0.296...",0.532750,"[0.7753036437246964, 0.4174917491749175, 0.405...",None,0.675658,...,1,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.628544,0.512789,"[0.7565938206480783, 0.4426162073220896, 0.339...",0.504722,"[0.7900535096002518, 0.4413453650533224, 0.282...",0.531128,"[0.7258530942741469, 0.4438943894389439, 0.423...",None,0.664579,...,12,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.626437,0.507183,"[0.7601255417725303, 0.4200542005420054, 0.341...",0.505546,"[0.7865759356634705, 0.46407185628742514, 0.26...",0.531808,"[0.7353961827646038, 0.38366336633663367, 0.47...",None,0.658960,...,123,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.565709,0.433717,"[0.7083589428395821, 0.373931623931624, 0.2188...",0.432990,"[0.7557377049180328, 0.32894736842105265, 0.21...",0.441125,"[0.6665702718334298, 0.43316831683168316, 0.22...",None,0.658157,...,1,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.574521,0.447132,"[0.7120467117393977, 0.38920454545454547, 0.24...",0.446022,"[0.759672131147541, 0.341645885286783, 0.23674...",0.455274,"[0.6700404858299596, 0.4521452145214521, 0.243...",None,0.661529,...,12,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.581609,0.460526,"[0.7260045489006822, 0.3461988304093567, 0.309...",0.454181,"[0.7631495058973542, 0.328159645232816, 0.2712...",0.472881,"[0.6923076923076923, 0.36633663366336633, 0.36]",None,0.682402,...,12,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.569732,0.439547,"[0.7192224622030238, 0.370757180156658, 0.2286...",0.436661,"[0.7708333333333334, 0.33832539142273654, 0.20...",0.449870,"[0.6740890688259109, 0.41006600660066006, 0.26...",None,0.658157,...,123,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.609962,0.503577,"[0.739392069125135, 0.4348527632109722, 0.3364...",0.495249,"[0.7925901422428051, 0.425414364640884, 0.2677...",0.530111,"[0.6928860613071139, 0.44471947194719474, 0.45...",None,0.665222,...,1,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.622222,0.503489,"[0.7563909774436092, 0.4229765013054831, 0.331...",0.499210,"[0.7879072681704261, 0.44751381215469616, 0.26...",0.525793,"[0.727299016772701, 0.400990099009901, 0.44909...",None,0.675658,...,12,focal,2,True,5,Conv1d,None,concatenation,64,23


In [23]:
best_seqsignet_attention_mha_umap_kfold_110["f1"].mean()

0.5000366546431735

In [24]:
best_seqsignet_attention_mha_umap_kfold_110["precision"].mean()

0.4950095452721604

In [25]:
best_seqsignet_attention_mha_umap_kfold_110["recall"].mean()

0.516300748254891

In [26]:
np.stack(best_seqsignet_attention_mha_umap_kfold_110["f1_scores"]).mean(axis=0)

array([0.75517386, 0.4064685 , 0.3384676 ])

In [27]:
np.stack(best_seqsignet_attention_mha_umap_kfold_110["precision_scores"]).mean(axis=0)

array([0.77258417, 0.43001749, 0.28242698])

In [28]:
np.stack(best_seqsignet_attention_mha_umap_kfold_110["recall_scores"]).mean(axis=0)

array([0.73857721, 0.38668867, 0.42363636])

## GRP

In [29]:
(
    seqsignet_attention_mha_grp_kfold_110,
    best_seqsignet_attention_mha_grp_kfold_110,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_attention_mha_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 36: history length = 110
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 36: history length = 110
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_grp_focal_2_3_5_36_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_mha_grp_focal_2_3_5_36_kfold_best_model.csv


In [30]:
seqsignet_attention_mha_grp_kfold_110

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.635824,0.508797,"[0.7639760618887753, 0.4242424242424242, 0.338...",0.507388,"[0.7712938402593575, 0.4694694694694695, 0.281...",0.522465,"[0.7567958357432042, 0.38696369636963696, 0.42...",None,0.675819,...,1,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.631034,0.514329,"[0.7607196401799101, 0.43474540008557977, 0.34...",0.507800,"[0.7898505603985057, 0.45155555555555554, 0.28...",0.535177,"[0.7336610757663389, 0.41914191419141916, 0.45...",None,0.654143,...,12,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.625670,0.516982,"[0.7487775061124694, 0.4594807238394965, 0.342...",0.507188,"[0.7939079714841218, 0.43909774436090226, 0.28...",0.537389,"[0.708502024291498, 0.48184818481848185, 0.421...",None,0.657836,...,123,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.598084,0.465473,"[0.7368264806293604, 0.37138389366692726, 0.28...",0.462382,"[0.7569380908813663, 0.35289747399702825, 0.27...",0.469890,"[0.7177559282822441, 0.3919141914191419, 0.3]",None,0.693802,...,1,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.617816,0.467523,"[0.7565293093441672, 0.37725701114099114, 0.26...",0.477892,"[0.7591729761211415, 0.35298346513299783, 0.32...",0.463310,"[0.753903990746096, 0.4051155115511551, 0.2309...",None,0.694605,...,12,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.633333,0.489539,"[0.7692972350230415, 0.38879456706281834, 0.31...",0.488852,"[0.7662076878944348, 0.40034965034965037, 0.3]",0.490706,"[0.7724117987275883, 0.3778877887788779, 0.321...",None,0.693642,...,12,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.608621,0.471848,"[0.7453325554259044, 0.3671811535337124, 0.303...",0.470239,"[0.7519128899352561, 0.3616, 0.2972027972027972]",0.473632,"[0.7388663967611336, 0.37293729372937295, 0.30...",None,0.699101,...,123,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.637548,0.486681,"[0.7714041095890412, 0.4019055868341273, 0.286...",0.488453,"[0.7614084507042254, 0.422971741112124, 0.2809...",0.485744,"[0.7816657027183342, 0.38283828382838286, 0.29...",None,0.672768,...,1,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.638314,0.507621,"[0.7690036900369003, 0.4336569579288026, 0.320...",0.502597,"[0.7853482062104311, 0.4253968253968254, 0.297...",0.514281,"[0.7533256217466744, 0.44224422442244227, 0.34...",None,0.666988,...,12,focal,2,True,5,Conv1d,None,concatenation,64,23


In [31]:
best_seqsignet_attention_mha_grp_kfold_110["f1"].mean()

0.5012734570184637

In [32]:
best_seqsignet_attention_mha_grp_kfold_110["precision"].mean()

0.49600739458541393

In [33]:
best_seqsignet_attention_mha_grp_kfold_110["recall"].mean()

0.5104409000761269

In [34]:
np.stack(best_seqsignet_attention_mha_grp_kfold_110["f1_scores"]).mean(axis=0)

array([0.74955195, 0.42238012, 0.3318883 ])

In [35]:
np.stack(best_seqsignet_attention_mha_grp_kfold_110["precision_scores"]).mean(axis=0)

array([0.77881209, 0.39856527, 0.31064482])

In [36]:
np.stack(best_seqsignet_attention_mha_grp_kfold_110["recall_scores"]).mean(axis=0)

array([0.72286485, 0.44966997, 0.35878788])